# Python Binding
## Henry Schreiner
## PyCHEP 2018


# Focus

* What Python bindings do
* What tools are available
* How Python bindings work

# Caviats

* Will cover C++ and C binding only
* Will not cover every tool available
* Will not cover `cppyy` in detail (but see next talk)
* Python 2 is dying, long live Python 3!

# Selection:
* C Binding: ctypes, <pypy based thing>
* Manual binding (quick look)
* SWIG (Too automatic)
* Cython (Two languages)
* PyBind11 (just right)
* Special mention: Numba, cppyy

# Preperation

This talk is interactive, and uses `cookiecutter` to set up code for you to play with. Install CookieCutter:

```shell
pip install cookiecutter
```

Then run:

```shell
cookiecutter gh:henryiii/pybindings_cc
```

Answer the questions, and then you'll have a new directory ready to go!

# What is it?

Bindings allow a function(alitiy) to be accessed from Python. 

### Source: simple.c
```c
float square(float x) {
    return x*x;
}
```

### Desired usage
```python
y = square(x)
```

# C Bindings

```cpp
cc simple.c -shared -o simple.so
```

C bindings are increably easy. Just compile into a shared library, then open it in python with the built in `ctypes` module:

```python
from ctypes import cdll, c_float
lib = ctypes.cdll.LoadLibrary('simple.so')
lib.square.argtypes = (ctypes.c_float,)
lib.square.restype = ctypes.c_float
lib.square(2.0) == 4.0
```

See [AmpGen](https://gitlab.cern.ch/lhcb/Gauss/blob/LHCBGAUSS-1058.AmpGenDev/Gen/AmpGen/options/ampgen.py) Python interface

# iOS example

Using Pythonista for iOS, we can use `ctypes` to access Apple's public APIs:

# Why do we need more?

For some cases, this is enougth. But often, you have an elegant (hopefully) C++ api. You can't reduce this down to `export "C"`. You'd like to be able to do the same thing in Python that you'd do in C++. You need memory managment, etc.

Let's start looking at ways to bind C++, but let's try one more C example first:

```c
#include <Python.h>

float square(float x) {
    return x*x; }

/* C reminder: static means only visible in this file (not exported) */
static PyObject* square_wrapper(PyObject* self, PyObject* args) {
  float input, result;
  PyObject* ret;

  if (!PyArg_ParseTuple(args, "f", &input)) {
      return NULL; }

  result = square(input);
  ret = PyFloat_FromDouble(result);
  return ret; }

static PyMethodDef PySimpleMethods[] = {
 { "square", square_wrapper, METH_VARARGS, "Square function" },
 { NULL, NULL, 0, NULL } };

DL_EXPORT(void) initpysimple(void) {
  Py_InitModule("pysimple", PySimpleMethods); }
```

Build:

```bash
cc -fPIC -I /usr/include/python2.7 -shared -o pysimple.so pysimple.c -L /usr/lib  -lpython2.7
```

Run:

```python
import pysimple
pysimple.square(2.0) == 4.0
```

In [2]:
!pip install cppyy

    100% |████████████████████████████████| 122kB 5.7MB/s ta 0:00:01
    100% |████████████████████████████████| 49.8MB 536kB/s eta 0:00:01
  Running setup.py bdist_wheel for cppyy ... done
  Stored in directory: /Users/henryiii/Library/Caches/pip/wheels/80/98/9f/771658b19903186a6ae15b024c1651d01032f98d8f199ad15b
  Running setup.py bdist_wheel for CPyCppyy ... done
  Running setup.py clean for CPyCppyy
  Running setup.py bdist_wheel for cppyy-backend ... done
  Running setup.py clean for cppyy-backend
  Running setup.py bdist_wheel for cppyy-cling ... done
  Stored in directory: /Users/henryiii/Library/Caches/pip/wheels/9c/cc/0a/bf008b9fc74a35714a3cfeca0e2774d9ef0ea281f8bdd71520
Successfully built cppyy cppyy-cling
Failed to build CPyCppyy cppyy-backend
distributed 1.21.8 requires msgpack, which is not installed.
tensorboard 1.8.0 has requirement bleach==1.5.0, but you'll have bleach 2.1.3 which is incompatible.
tensorboard 1.8.0 has requirement html5lib==0.9999999, but you'll have htm

In [3]:
import cppyy